# 0. Чтение данных и базовое ознакомление

In [82]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [83]:
# считываем данные
df = pd.read_csv('data/data.csv')

In [84]:
# выясним размерность датасета
print('размерность датасета: ',df.shape)
# проверим корректность загрузки и ознакомимся с полями
df.head()

размерность датасета:  (377185, 18)


,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [85]:
# оценим количество пропусков и типы данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [86]:
# посмотрим на количество явных пропусков с более удобного ракурса
df.isnull().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

Описание данных:

- 'status' — статус продажи;
- 'private pool' и 'PrivatePool' — наличие собственного бассейна;
- 'propertyType' — тип объекта недвижимости;
- 'street' — адрес объекта;
- 'baths' — количество ванных комнат;
- 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
- 'fireplace' — наличие камина;
- 'city' — город;
- 'schools' — сведения о школах в районе;
- 'sqft' — площадь в футах;
- 'zipcode' — почтовый индекс;
- 'beds' — количество спален;
- 'state' — штат;
- 'stories' — количество этажей;
- 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
- 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

_____________

# 1. Предобработка данных

In [87]:
# для начала удалим записи с пустыми значениями целевой переменной, они явно не участвуют в этом мероприятии
df = df[~df['target'].isna()]
df.shape[0]

374704

In [88]:
# пройдемся по датасету и уберем общую проблему - лишние пробелы и переносы строк
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

In [89]:
# проанализируем датасет на наличие полных дубликатов
df.duplicated().sum()

49

In [90]:
# удалим полные дубликаты исходного датасета
df = df.drop_duplicates()
df.shape[0]

374655

#### Последовательно проверим признаки

In [91]:
# ознакомимся с внесенными значениями по диагонали, чтобы выделить основные сложности с обработкой
columns = df.columns
for column in columns:
    print()
    print(column)
    print('количество вариантов : ', df[column].value_counts().shape[0])
    print()
    print(df[column].value_counts().head(50))
    if df[column].value_counts().shape[0] > 50:
        print(df[column].value_counts().tail(50))


status
количество вариантов :  156

for sale                             156054
Active                               105206
For sale                              43464
foreclosure                            5677
New construction                       5458
Pending                                4697
Pre-foreclosure                        2000
P                                      1488
Pre-foreclosure / auction              1281
Under Contract Show                    1183
/ auction                               799
Under Contract   Showing                793
Active Under Contract                   718
New                                     690
Under Contract                          690
Contingent                              581
Price Change                            563
Auction                                 493
A Active                                443
for rent                                398
Foreclosure                             343
Foreclosed                             

##### Первичный осмотр признаков

- status

156 вариантов - возможно просмотреть и обработать детально. 
Какие-то категории точно можно объединять, вроде “Coming soon”
При этом, объект может менять свои статусы, и какие-то могут влиять на цену (например, как предположение, за долги), а какие-то нет (когда проходит обычный жизненный цикл вроде такого: “для продажи - показы - договор без обязательств”)

- propertyType

Один из самых сложных признаков в выборке. 
Разное написание, разные сокращения, и перечисление признаков через запятые и слэш, а где-то и просто через пробел. Очень неудобно.
Даже при разложении каждой записи этого поля на отдельные составляющие вложенного списка пока конечное количество вариантов непонятно.

- street

нет цели проверить каждый адрес, но топ повторяющихся вариантов можно и нужно просмотреть как минимум на предмет разных написаний “адреса нет”
Навскидку по крайней мере похоже, что в поле и правда вносят адрес.

- baths

Вариантов 226. Многовато, но можно обработать массовые нюансы - убрать типовые наборы слов в начале или конце записи, после чего повторно посмотреть схлопнувшиеся варианты и доработать еще раз.
Уже видны записи с 76 и 241 ванной. Пока тяжело осознать, что это может быть, если  не выброс. Также присутствуют записи через слэш. Надо смотреть подробнее.

- homefacts

Пока только понятно, что надо разложить на отдельные поля этот вложенный список. После этого уже отдельно смотреть что оттуда вытянется.

- fireplace

Разное написание, и где-то цифры, где-то слова. 1653 варианта - много для полноценной ручной обработки с индивидуальным просмотром.
При этом, наблюдаются характерные записи с указанием, газ это, электрика или на дровах/брикетах.
Нужно уменьшать количество вариантов.

- city

2019 нас.пункта. В первом приближении похоже, что внесены города.
Есть вероятность разного написания, но это можно выяснить только при более детальном просмотре.

- schools

списки разной длины, с разным количеством школ и соответствующих им характеристик. Здесь вариант только приводить к какой-то средней оценке. 

- sqft

Площадь объекта - по ощущениям основной показатель, его следует обработать максимально аккуратно.
Видно, что присутствует различное написание единиц измерения, а где-то ЕИ совсем отсутствует. Где-то есть разделитель разрядов, где-то его нет. Есть отдельные записи и с текстовым описанием. 
И конечно, критичны записи с отсутствием информации. Учитывая неаккуратность датасета, эта информация может присутствовать в других полях, необходимо посмотреть подробнее.

- zipcode 

Порадовало, что в верхней части рейтинга популярности значений отсутствует “нет кода” или нулевое значение. Этот факт дает надежду, что сможем определять нахождение объекта более подробно, нежели штат или город. 
Но есть некие значения через дефис. Первая составная часть похожа на основную массу индексов, со второй (после дефиса) пока непонятно.

- beds

1147 вариантов. Разное написание/обозначение слова “кровать”. Но что удивительно, в этом поле кое-где внесены значения площади. Возможно, в форме для заполнения какие-то поля были рядом и неочевидно подписаны, за счет чего сюда вносилась информация о площади (то ли жилой, то ли участка). Надо смотреть.

- state

Выглядит аккуратно, но смущают единичные значения - неужели один объект во всем штате? Надо проверить.

- stories

Знаки плюсов, где-то значения с точками, где-то без точек, где-то словами вроде “One” и т.п. Чистить и смотреть результат.

- mls-id и MlsId

Много повторяющихся значений, хотя по описанию признака казалось, что это уникальный идентификатор. Похоже, что нет, надо посмотреть поближе.

- target

Собственно целевой признак. Есть со значком $, есть без него. Есть плюсы, которые даже если что-то обозначали, мы проигнорируем и удалим.
И бросается в глаза в топ-30 значение $1000 в количестве почти тысяча записей. Тоже надо посмотреть.


Приступим к обработке от простого к сложному

____________

##### Частный бассейн - 'private pool' и 'PrivatePool'

In [92]:
# проверим пару признаков наличия частного бассейна
# для начала проверим варианты внесения информации
display(df['private pool'].value_counts())
display(df['PrivatePool'].value_counts())

Yes    4151
Name: private pool, dtype: int64

yes    28686
Yes    11434
Name: PrivatePool, dtype: int64

In [93]:
# предположим, что это признаки, созданные в базе данных в разное время
# т.е., информация о наличии бассейна есть либо в одном, либо в другом
# в таком случае, не должно быть записей, в которых одновременно указано наличие в обоих полях
# проверим простым способом - переведем наличие в "1" и сложим оба столбца
# если запись и там, и там, получим в этих записях двойки

# для начала переведем записи в единицы и проверим сохранение количеств значений
df['PrivatePool'] = df['PrivatePool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['PrivatePool'].value_counts())
df['private pool'] = df['private pool'].apply(lambda x: 1 if x in ['yes', 'Yes'] else 0)
display(df['private pool'].value_counts())

0    334535
1     40120
Name: PrivatePool, dtype: int64

0    370504
1      4151
Name: private pool, dtype: int64

In [94]:
# поскольку перекодирование столбцов прошло успешно, теперь делаем аггрегирующее поле
# проверка вариантов значений покажет, подтвердилось ли вышеописанное предположение
df['PoolPrivate'] = df['PrivatePool'] + df['private pool']
df['PoolPrivate'].value_counts()

0    330384
1     44271
Name: PoolPrivate, dtype: int64

In [95]:
# схема сработала, изначальные признаки можно удалить
df = df.drop(columns=['private pool', 'PrivatePool'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  mls-id        24937 non-null   object
 14  MlsId         310187 non-null  object
 15  target        374655 non-null  object
 16  PoolPrivate   374655 non-null  int64 
dtypes: int64(1), object(16)
memory usage: 51.5+ MB


__________

##### Идентификаторы MLS - 'mls-id' и 'MlsId'

In [96]:
# проверим пару признаков идентификаторов MLS
# предположение такое - признаки заполнялись в разные периоды времени (как бассейны)
# если это так, значение есть либо в одном поле, либо в другом, проверим
print('Количество записей с одновременно заполненными полями "MlsId" и "mls-id" : ',df[~df['MlsId'].isna() & ~df['mls-id'].isna()].shape[0])

Количество записей с одновременно заполненными полями "MlsId" и "mls-id" :  0


In [97]:
# нет ни одной записи, в которой одновременно были бы непустые значения в обоих полях
# теперь проверим, нет ли одинаковых идентификаторов в обоих полях, и если есть, то идентичные ли записи им соответствуют
# это будет обозначать, что в какой-то момент одно поле для внесения отключили, а второе активировали
# для начала переведем в строчные буквы
df['MlsId'] = df['MlsId'].str.lower()
df['mls-id'] = df['mls-id'].str.lower()
MlsIdList = list(df['MlsId'].unique())
print('уникальных значений MlsId : ',len(MlsIdList))
mls_id_List = list(df['mls-id'].unique())
print('уникальных значений mls-id : ',len(mls_id_List))
print ('при пересечении только nan длина множества должна быть : ', (len(MlsIdList)+len(mls_id_List)-1))
Mls = set(MlsIdList + mls_id_List)
print('уникальных значений кумулятивно по обоим столбцам : ',len(Mls))
if len(Mls) < (len(MlsIdList)+len(mls_id_List)-1):
    print('значения столбцов пересекаются')
else:
    print('значения столбцов не пересекаются')

уникальных значений MlsId :  232861
уникальных значений mls-id :  24902
при пересечении только nan длина множества должна быть :  257762
уникальных значений кумулятивно по обоим столбцам :  248925
значения столбцов пересекаются


In [98]:
# выделим несколько ID MLS, которые присутствуют в обоих полях, и проверим, одинаковые ли объекты им соответствуют
q = list(set(MlsIdList) & set(mls_id_List))[1:]
df[df['mls-id'].isin(q[:10]) | df['MlsId'].isin(q[:10])].sort_values(by='street')

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,PoolPrivate
155152,For sale,Condo,1020 NE 63rd St # 2,2 ba,"{'atAGlanceFacts': [{'factValue': '1977', 'fac...",NaN,Vancouver,"[{'rating': ['4/10', '4/10', '4/10'], 'data': ...","1,206 sqft",98665,2 bd,WA,NaN,19033569,NaN,"$199,000",0
41449,for sale,condo,1020 NE 63rd St #2,2 Baths,"{'atAGlanceFacts': [{'factValue': '1977', 'fac...",yes,Vancouver,"[{'rating': ['4/10', '4/10', '4/10'], 'data': ...","1,206 sqft",98665,2 Beds,WA,2.0,NaN,19033569,"$199,000",0
367808,Foreclosure,Single Family,13624 Carlton Pl,Bathrooms: 1,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",NaN,Flushing,"[{'rating': ['8/10', '9/10', '2/10'], 'data': ...",Total interior livable area: 864 sqft,11354,2 bd,NY,2.0,10630802,NaN,"$425,000",0
203980,foreclosure,single-family home,13624 Carlton Pl,NaN,"{'atAGlanceFacts': [{'factValue': '1925', 'fac...",NaN,Flushing,"[{'rating': ['2/10', '9/10', '8/10'], 'data': ...",864 sqft,11354,2 Beds,NY,2.0,NaN,10630802,"$425,000",0
254639,for sale,townhouse,2205 Chunk Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",yes,Dallas,"[{'rating': ['5/10', '5/10', '5/10'], 'data': ...","2,629 sqft",75206,4 Beds,TX,NaN,NaN,14141471,"$575,000",0
207376,For sale,Townhouse,2205 Chunk Ct,4 ba,"{'atAGlanceFacts': [{'factValue': '2015', 'fac...",NaN,Dallas,"[{'rating': ['5/10', '5/10', '5/10'], 'data': ...","2,629 sqft",75206,4 bd,TX,NaN,14141471,NaN,"$585,000",0
323622,Active,Single Detached,2822 Dogwood Park Dr,NaN,"{'atAGlanceFacts': [{'factValue': '1951', 'fac...",NaN,Richland Hills,"[{'rating': ['5', '4', '7'], 'data': {'Distanc...","1,046",76118,NaN,TX,NaN,NaN,14210333,"157,000",0
322449,For sale,Single Family,2822 Dogwood Park Dr,Bathrooms: 3,"{'atAGlanceFacts': [{'factValue': '1951', 'fac...",NaN,Richland Hills,"[{'rating': ['5/10', '4/10', '7/10'], 'data': ...","Total interior livable area: 1,046 sqft",76118,3 bd,TX,1.0,14210333,NaN,"$157,000",0
99425,New construction,Single Family,290 Hillside Ave,NaN,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Philadelphia,"[{'rating': ['4/10', '1/10'], 'data': {'Distan...",NaN,19128,4 bd,PA,NaN,paph850088,NaN,"$699,000",0
156185,for sale,single-family home,290 Hillside Ave,5 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '4/10'], 'data': {'Distan...","3,045 sqft",19128,5 Beds,PA,3.0,NaN,paph850088,"$699,000",0


In [99]:
# на примере выборочной проверки делаем вывод, что идентификаторы в обоих полях относятся к одним и тем же объектам
# объединим идентификатор MLS в один столбец и удалим два исходных
df['MLS'] = df['MlsId'].fillna('') + df['mls-id'].fillna('')
df = df.drop(columns=['MlsId', 'mls-id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374655 entries, 0 to 377184
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        335399 non-null  object
 1   propertyType  340101 non-null  object
 2   street        374653 non-null  object
 3   baths         269308 non-null  object
 4   homeFacts     374655 non-null  object
 5   fireplace     102519 non-null  object
 6   city          374621 non-null  object
 7   schools       374655 non-null  object
 8   sqft          334560 non-null  object
 9   zipcode       374655 non-null  object
 10  beds          283726 non-null  object
 11  state         374655 non-null  object
 12  stories       224902 non-null  object
 13  target        374655 non-null  object
 14  PoolPrivate   374655 non-null  int64 
 15  MLS           374655 non-null  object
dtypes: int64(1), object(15)
memory usage: 48.6+ MB


In [100]:
# посмотрим неоднократно встречающиеся варианты для проверки разного написания отсутствия номера
df['MLS'].value_counts().head(50)

                             39531
no mls                          42
no mls #                        16
a, houston, tx 77008            13
no                              12
12a, orlando, fl 32833          11
b, houston, tx 77008             9
1, south boston, ma 02127        9
b, houston, tx 77007             8
2, washington, dc 20002          8
11a, orlando, fl 32833           8
1, washington, dc 20002          7
2, washington, dc 20010          7
1, washington, dc 20010          6
1, washington, dc 20001          6
a, austin, tx 78721              6
2, washington, dc 20001          6
a, austin, tx 78704              6
2101941                          6
1412350                          6
2088662                          6
3a, orlando, fl 32833            6
1a, orlando, fl 32833            6
2, washington, dc 20009          6
nomlsid                          6
2, boston, ma 02129              6
0, doral, fl 33178               6
a, houston, tx 77018             5
14181176            

In [101]:
# заменим обозначение отсутствия номера MLS на однотипное пустое значение
no_mls = ['no mls', 'no mls #', 'no', 'nomlsid']
df['MLS'] = df['MLS'].apply(lambda x: '' if (x in no_mls) else x)
df['MLS'].value_counts().head(50)

                             39607
a, houston, tx 77008            13
12a, orlando, fl 32833          11
1, south boston, ma 02127        9
b, houston, tx 77008             9
2, washington, dc 20002          8
11a, orlando, fl 32833           8
b, houston, tx 77007             8
2, washington, dc 20010          7
1, washington, dc 20002          7
2, boston, ma 02129              6
1, washington, dc 20010          6
2, washington, dc 20001          6
0, doral, fl 33178               6
a, austin, tx 78704              6
1412350                          6
1, washington, dc 20001          6
3a, orlando, fl 32833            6
a, austin, tx 78721              6
2, washington, dc 20009          6
2088662                          6
1a, orlando, fl 32833            6
2101941                          6
b, houston, tx 77057             5
1026004                          5
2281272                          5
1019437                          5
14168541                         5
1367153             

In [102]:
# оценим количество пропусков
print(df['MLS'].isna().sum())
print(round((df['MLS'].isna().sum()/df.shape[0]*100),0),'%')

0
0.0 %


____________

##### Целевой признак target

In [103]:
# оценим количество пропусков и варианты значений
print(df['target'].isna().sum())
print(round((df['target'].isna().sum()/df.shape[0]*100),0),'%')
df['target'].value_counts()

0
0.0 %


$225,000     1462
$275,000     1355
$250,000     1312
$350,000     1296
$299,900     1276
             ... 
274,359         1
$273,490+       1
$645,000+       1
$28,272         1
$171,306        1
Name: target, Length: 43939, dtype: int64

In [104]:
# проверим, нет ли помимо явных символов еще и букв в каких-то значениях
df[df['target'].str.contains('[a-zA-Z:]')]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,PoolPrivate,MLS
547,for rent,single-family home,4323 N Central Park Ave,3.5 Baths,"{'atAGlanceFacts': [{'factValue': '1913', 'fac...",yes,Chicago,"[{'rating': ['1/10', '4/10', '2/10', 'None/10'...","3,300 sqft",60618,4 Beds,IL,NaN,"$5,500/mo",0,10588057
609,for rent,multi-family,220 Boylston St #1412,2 Baths,"{'atAGlanceFacts': [{'factValue': '1985', 'fac...",yes,Boston,"[{'rating': [], 'data': {'Distance': [], 'Grad...","1,673 sqft",2116,2 Beds,MA,NaN,"$10,500/mo",0,72580936
2075,for rent,single-family home,2830 NE 56th Ct,4 Baths,"{'atAGlanceFacts': [{'factValue': '1965', 'fac...",NaN,Fort Lauderdale,"[{'rating': ['6/10', '2/10', '4/10'], 'data': ...","2,400 sqft",33308,4 Beds,FL,NaN,"$6,390/mo",1,a10521855
3025,for rent,multi-family,411 Kline Aly,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,Clarksville,"[{'rating': ['8/10', '9/10', '7/10'], 'data': ...","1,280 sqft",37040,2 Beds,TN,NaN,"$1,200/mo",0,2102821
3645,for rent,multi-family,240 E Illinois St #2011,2 Baths,"{'atAGlanceFacts': [{'factValue': '2003', 'fac...",NaN,Chicago,"[{'rating': ['4/10', '7/10'], 'data': {'Distan...","1,473 sqft",60611,2 Beds,IL,NaN,"$3,600/mo",1,10590275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371791,for rent,multi-family,9436 Turrentine Dr,1.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,El Paso,"[{'rating': ['4/10', '8/10', '6/10'], 'data': ...","1,050 sqft",79925,2 Beds,TX,NaN,$890/mo,0,820163
372459,for rent,townhouse,34 Jonquil Pl,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '2014', 'fac...",NaN,The Woodlands,"[{'rating': ['5/10', '8/10', '7/10', '8/10'], ...","2,601 sqft",77375,3 Beds,TX,NaN,"$2,500/mo",0,62158637
374288,for rent,single-family home,8864 Devonshire Dr,2 Baths,"{'atAGlanceFacts': [{'factValue': '2016', 'fac...",yes,Fort Worth,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","2,000 sqft",76131,4 Beds,TX,NaN,"$2,000/mo",0,
375550,for rent,townhouse,2217 W Seybert St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': ['1/10', '3/10'], 'data': {'Distan...",720 sqft,19121,2 Beds,PA,NaN,"$1,500/mo",0,paph857944


In [105]:
# обнаружились записи со стоимостью аренды в месяц, посмотрим в целом имеющие отношение к аренде записи
df[df['status'].str.contains('rent', na=False)]['status'].value_counts()

for rent              398
Apartment for rent      7
Condo for rent          7
Name: status, dtype: int64

In [106]:
# поскольку в целом модель должна предказывать стоимость продажи объекта
# плюс поскольку даже при желании на 400 записях достойный прогноз не построишь
# и эти записи составляют 0,1% от общего количества
# удаляем все записи, связанные с арендой
df = df[~df['status'].str.contains('rent', na=False)]
df.shape[0]

374243

In [107]:
# теперь заменим все остальные обнаруженные знаки, которые мешают перевести суммы в числовой формат
df['target'] = df['target'].apply(lambda x: int(x.replace('$','').
                                replace('+','').
                                replace(',','')))
# и собственно сменим тип данных в этом признаке
df['target'] = df['target'].astype(int)

________________

##### Рассмотрим штат - State

In [108]:
# оценим количество пропусков и варианты значений
print(df['state'].isna().sum())
print(round((df['state'].isna().sum()/df.shape[0]*100),0),'%')
df['state'].value_counts()

0
0.0 %


FL    114548
TX     83263
NY     24324
CA     23169
NC     21760
TN     18217
WA     13721
OH     12422
IL      8821
NV      8401
GA      6628
CO      6371
PA      5493
MI      5119
DC      4580
AZ      3347
IN      3279
OR      2774
MA      1493
UT      1319
MD      1086
VT       864
MO       832
VA       800
WI       452
NJ       436
ME       258
IA       242
KY        90
OK        49
MS        40
SC        28
MT         7
DE         5
Fl         1
BA         1
AL         1
OT         1
OS         1
Name: state, dtype: int64

In [109]:
# в предположении, что индекс должен относиться только к одному штату, проверим соответствие индексов и штатов
# выделим те индексы, к которым сопоставлены более 1 штата
states = df.groupby(['zipcode', 'state'])['status'].agg('count').reset_index()[['zipcode', 'state']]
states1 = states.groupby('zipcode').agg('count').reset_index()
states2 = states1[states1['state'] > 1]
states2

,zipcode,state
0,--,2
1,0,3
197,11210,2
538,20003,2
1482,33179,2
1512,33321,2
2997,77380,2
3050,77710,2
3155,78501,2
3956,92703,2


In [110]:
# зафиксируем перечень индексов, к которым сопоставлены несколько штатов
states3 = list(states2['zipcode'][2:])
states3

['11210', '20003', '33179', '33321', '77380', '77710', '78501', '92703']

In [111]:
# теперь по выбранному перечню проверим соотношение количества записей, относящихся к тому или иному штату
df[df['zipcode'].isin(states3)].pivot_table('target', 'zipcode','state', 'count','')

state,BA,CA,DC,DE,FL,Fl,MA,NY,OT,TN,TX
zipcode,,,,,,,,,,,
11210,,,,,,,,194.0,1.0,,
20003,,,188.0,,,,1.0,,,,
33179,1.0,,,,682.0,,,,,,
33321,,,,,561.0,1.0,,,,,
77380,,,,,,,,,,1.0,124.0
77710,,,,3.0,,,,,,,5.0
78501,,,,,2.0,,,,,,165.0
92703,,84.0,,,,,,,,2.0,


In [112]:
# несоответствия очевидны (кроме 77710, интернет отнес к TX Техас), делаем замены
zipcode_replace = {'11210': 'NY',
                   '20003': 'DC',
                   '33179': 'FL',
                   '33321': 'FL',
                   '77380': 'TX',
                   '77710': 'TX',
                   '78501': 'TX',
                   '92703': 'CA'}

In [113]:
# напишем и применим функцию для замены
def zip_change(zip, state):
    if zip in zipcode_replace.keys():
        return zipcode_replace.get(zip)
    else:
        return state
    
df['state'] = df.apply(lambda row: zip_change(row['zipcode'], row['state']), axis=1)

In [114]:
# проверим себя, повторно проведем те же операции по проверке соответствия одному индексу одного штата
states = df.groupby(['zipcode', 'state'])['status'].agg('count').reset_index()[['zipcode', 'state']]
states1 = states.groupby('zipcode').agg('count').reset_index()
states2 = states1[states1['state'] > 1]
states2

,zipcode,state
0,--,2
1,0,3


In [115]:
# дополнительно отнесем в группу Other штаты, в которых количество записей менее 100
other = ['KY', 'OK', 'MS', 'SC', 'MT', 'DE', 'Fl', 'BA', 'AL', 'OT', 'OS']
df['state'] = df['state'].apply(lambda x: 'Other' if x in other else x)
df['state'].value_counts()

FL       114548
TX        83269
NY        24325
CA        23171
NC        21760
TN        18214
WA        13721
OH        12422
IL         8821
NV         8401
GA         6628
CO         6371
PA         5493
MI         5119
DC         4581
AZ         3347
IN         3279
OR         2774
MA         1492
UT         1319
MD         1086
VT          864
MO          832
VA          800
WI          452
NJ          436
ME          258
IA          242
Other       218
Name: state, dtype: int64

_____________

##### City - город

In [116]:
# оценим количество пропусков
print(df['city'].isna().sum())
print(round((df['city'].isna().sum()/df.shape[0]*100),0),'%')

34
0.0 %


In [117]:
# в предположении, что индекс должен относиться только к одному городу, проверим соответствие индексов и городов
# выделим те индексы, к которым сопоставлены более 1 значения города
df['city'] = df['city'].str.lower()
cities = df.groupby(['zipcode', 'city'])['target'].agg('count').reset_index()[['zipcode', 'city']]
cities1 = cities.groupby('zipcode').agg('count').reset_index()
cities2 = cities1[cities1['city'] > 1][2:].reset_index().drop('index', axis=1)
cities2

,zipcode,city
0,02119,2
1,02122,2
2,02124,3
3,02125,2
4,02127,2
...,...,...
991,98908,2
992,99206,2
993,99208,2
994,99212,3


Достаточно много разночтений для индивидуального просмотра. Массово идей нет,  
т.к. к разным индексам привязаны разные лидеры написания одного и того же города.  
Пока оставляем как есть.

In [118]:
#zip_db = pd.read_csv('data/zip_code_database.csv', dtype='str')
#zip_db.head()

______________

##### street - адрес объекта

In [119]:
# посмотрим на пропуски и на предмет повторяющихся формулировок
print(df['street'].isna().sum())
print(round((df['street'].isna().sum()/df.shape[0]*100),0),'%')
df['street'].value_counts().head(50)

2
0.0 %


Address Not Disclosed            672
Undisclosed Address              516
(undisclosed Address)            391
Address Not Available            175
Unknown Address                   72
2103 E State Hwy 21               57
11305 Gulf Fwy                    54
17030 Youngblood Rd.              38
NE 58th Cir                       34
9470 Lancaster Rd. SW             32
1 Palmer Dr                       27
8426 Terrace Valley Circle        25
9845 Basil Western Rd NW          25
6320 SW 89th Court Road           24
8447 SW 99th Street Rd            22
5221 S. Zapata Hwy                20
Whitetail Trail                   19
Stone Bluff Drive                 18
2005 West Happy Valley Road       17
3435 Heather Garden Trail         17
Boncher Blvd                      17
1365 Neihart Way                  17
3423 Heather Garden Trail         17
1727 Opal Field Lane              17
24423 Ferdossa Drive              17
50 Leanni Way                     16
606 Vineyard Hollow Court         16
1

In [120]:
# заменим разные вариации отсутствия адреса на единое значение
no_street = ['Address Not Disclosed', 
             'Undisclosed Address', 
             '(undisclosed Address)', 
             'Address Not Available', 
             'Unknown Address']
df['street'] = df['street'].apply(lambda x: 'no address' if x in no_street else x).fillna('no address')
df['street'].value_counts()

no address                 1828
2103 E State Hwy 21          57
11305 Gulf Fwy               54
17030 Youngblood Rd.         38
NE 58th Cir                  34
                           ... 
1346 Midland Ave APT 3J       1
302 Tempranillo Way           1
8288 Mount Nido Dr            1
25230 Cadiz Dr                1
7810 Pereida St               1
Name: street, Length: 298644, dtype: int64

По хорошему надо извлечь координаты с помощью библиотеки Geopy, но в данном случае технические ресурсы не позволяют это сделать.

___________

##### baths - количество ванных комнат

In [121]:
# посмотрим пропуски и разнообразие значений для составления плана работ
print(df['baths'].isna().sum())
print(round((df['baths'].isna().sum()/df.shape[0]*100),0),'%')
df['baths'].unique()

105250
28.0 %


array(['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3',
       'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5',
       '1,000', '7 Baths', '2.0', '3.0', 'Bathrooms: 1', '4.0',
       '2.1 Baths', '2.5 Baths', '1', 'Bathrooms: 3', '4.5', '6 Baths',
       'Bathrooms: 4', '3 ba', '5', '2,500', '5.5 Baths', '1.0',
       '5 Baths', '1.5', '4', '~', '2.5', '4,000', '3.5 Baths', '2,000',
       '3,000', '8.0', '1 ba', '0', '5.0', '1,500', '7.0', '1,250',
       '9 Baths', '2,250', '6.0', '12 Baths', '5.5', '3,500', '1.5 Baths',
       '2,750', 'Bathrooms: 6', '4.5 Baths', '750', '5.5+', '6',
       '10 Baths', '6 ba', 'Bathrooms: 19', '10.0', '4 ba', '12 ba',
       '2.5+', '8', '7.5+', 'Bathrooms: 10', '0 / 0', 'Sq. Ft.', '5 ba',
       '4.5+', '18 Baths', '-- baths', 'Bathrooms: 7', '7', '18', '3.5+',
       '1.5+', '11 Baths', '5,000', '1.75 Baths', '9', '12.0', '6.5',
       'Bathrooms: 8', '10', '19 Baths', 'Bathrooms: 9', '16 Baths',
       '13 Baths', 'Bathrooms: 13'

In [122]:
# проблему составляют буквы, пробелы и двоеточия, а также запятые в качестве разделителя. Произведем замену
df['baths'] = df['baths'].str.replace('Sq. Ft.','')
df['baths'] = df['baths'].str.replace('[a-zA-Z]','', regex=True)
df['baths'] = df['baths'].str.replace(' ','')
df['baths'] = df['baths'].str.replace(':','')
df['baths'] = df['baths'].str.replace('+','')
df['baths'] = df['baths'].str.replace('~','')
df['baths'] = df['baths'].str.replace(',','.')
df['baths'] = df['baths'].str.replace('2-1/2-1/1-1/1-1','4')
df['baths'] = df['baths'].str.replace('1/1-0/1-0/1-0','1')
df['baths'] = df['baths'].str.replace('1-0/1-0/1','1')
df['baths'] = df['baths'].str.replace('1/1/1/1','4')
df['baths'] = df['baths'].str.replace('3-1/2-2','3')
df['baths'] = df['baths'].str.replace('0/0','0')
df['baths'] = df['baths'].str.replace('116/116/116','116')
df['baths'] = df['baths'].str.replace('--','')
df['baths'] = df['baths'].str.replace('—','')
df['baths'] = df['baths'].apply(lambda x: 0 if x=='' else x)
df['baths'] = df['baths'].astype(float)
df['baths'].value_counts()

/var/folders/0v/d10yh6tj1mn387jzy55lmxv80000gn/T/ipykernel_2847/3676306326.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['baths'] = df['baths'].str.replace('Sq. Ft.','')


/var/folders/0v/d10yh6tj1mn387jzy55lmxv80000gn/T/ipykernel_2847/3676306326.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['baths'] = df['baths'].str.replace('+','')


2.0      102613
3.0       66381
4.0       26229
1.0       17520
2.5       13687
          ...  
14.5          1
5.2           1
116.0         1
35.0          1
68.0          1
Name: baths, Length: 81, dtype: int64

In [123]:
df[df['baths'].isna()]['propertyType'].value_counts()

lot/land                                                       19917
single-family home                                             10246
Land                                                            7211
condo                                                           6635
Traditional                                                     4262
                                                               ...  
2 Stories, Colonial                                                1
Other - See Remarks                                                1
Single Detached, Vacation Home                                     1
2 Unit Condo                                                       1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1035, dtype: int64

In [124]:
df[df['propertyType']=='single-family home']['baths'].value_counts()

2.00     32149
3.00     24610
4.00     10446
5.00      4083
2.50      3408
6.00      1972
3.50      1423
7.00       980
8.00       476
4.50       345
9.00       262
1.50       134
10.00      129
5.50        79
11.00       71
1.75        70
12.00       44
2.10        42
2.75        32
13.00       28
6.50        17
14.00       13
3.10        10
1.10         9
2.25         9
3.75         8
15.00        7
3.25         7
1.25         6
16.00        4
17.00        3
8.50         3
4.25         2
3.20         2
4.10         2
7.50         2
20.00        2
2.20         2
19.00        2
25.00        1
4.75         1
23.00        1
5.25         1
5.20         1
22.00        1
42.00        1
11.50        1
24.00        1
26.00        1
30.00        1
27.00        1
6.75         1
0.75         1
Name: baths, dtype: int64

Пока не до конца понятно, что делать с пропусками. Пройдем по остальным признакам и вернемся.

_______________

##### Stories - количество этажей

In [125]:
# оценим пропуски и посмотрим варианты значений
print(df['stories'].isna().sum())
print(round((df['stories'].isna().sum()/df.shape[0]*100),0),'%')
df['stories'].unique()

149343
40.0 %


array([nan, '2.0', '1.0', '3.0', 'One', '2', 'Multi/Split', '4.0', '0.0',
       '0', 'One Level', '1', '9.0', '3', '1 Level, Site Built',
       'One Story', '3.00', '1.00', '14.0', 'Two', '3+', '1 Story', '5.0',
       '2 Story', 'Ranch/1 Story', 'Condominium', 'Stories/Levels', '7.0',
       '2 Level, Site Built', '2 Level', '15', '3 Level, Site Built', '4',
       '22.0', '2.00', '6.0', '1.0000', 'Lot', '3 Story', 'Three Or More',
       '1.5', '1 Level', 'Two Story or More', 'Site Built, Tri-Level',
       '54.0', '23', 'Farm House', '8.0', '16.0', '1.50', '18', '9', '21',
       '8', '12.0', 'Split Level w/ Sub', '11.0', '1.5 Stories', '7',
       '11', 'Townhouse', '12', '21.0', '16', '1.5 Story/Basement',
       '28.0', 'Traditional', '2.5 Story', '17', '2.0000', '63.0',
       'Acreage', 'Ground Level, One', '6', 'Split Foyer', '2 Stories',
       '27.0', '19.0', '2.50', '1.30', '2 Story/Basement', 'Split Level',
       '1.5 Story', '1.5 Level', '2 Or More Stories',
       '1 

In [126]:
df[df['stories'].isna()]['propertyType'].value_counts()

single-family home                                             18973
lot/land                                                       18566
condo                                                          10950
Land                                                           10308
Single Family Home                                              9370
                                                               ...  
Single Detached, Contemporary/Modern, Southwestern                 1
Traditional, Texas Hill Country                                    1
Historical/Conservation District, Single Detached, Colonial        1
Bungalow, Transitional                                             1
Bilevel, Converted Dwelling, Loft with Bedrooms, Condo/Unit        1
Name: propertyType, Length: 1007, dtype: int64

Данный признак достойно не обработать, т.к. типы объектов недвижимости, для которых отсутствует  
информация об этажности, имеют большую вариативность количества этажей.  
То есть, на основании типа объекта сделать выводы об этажности не представляется возможным.  
Придется удалить признак.

In [128]:
df = df.drop('stories', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374243 entries, 0 to 377184
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        334987 non-null  object 
 1   propertyType  339689 non-null  object 
 2   street        374243 non-null  object 
 3   baths         268993 non-null  float64
 4   homeFacts     374243 non-null  object 
 5   fireplace     102425 non-null  object 
 6   city          374209 non-null  object 
 7   schools       374243 non-null  object 
 8   sqft          334168 non-null  object 
 9   zipcode       374243 non-null  object 
 10  beds          283361 non-null  object 
 11  state         374243 non-null  object 
 12  stories       224900 non-null  object 
 13  target        374243 non-null  int64  
 14  PoolPrivate   374243 non-null  int64  
 15  MLS           374243 non-null  object 
dtypes: float64(1), int64(2), object(13)
memory usage: 48.5+ MB


___________

##### zipcode - почтовый индекс

In [129]:
# косвенно встреча с индексами была ранее при обработке признаков city и state
# оценим пропуски и посмотрим варианты значений
print(df['zipcode'].isna().sum())
print(round((df['zipcode'].isna().sum()/df.shape[0]*100),0),'%')

0
0.0 %


In [130]:
# посмотрим некорректные индексы
df['zipcode'].sort_values().head(10)

235207       --
231282       --
30261         0
83522         0
305572        0
308229    00000
10837     02108
169423    02108
123565    02108
293976    02108
Name: zipcode, dtype: object

In [131]:
df[(df['zipcode']=='--') | (df['zipcode']=='0')]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,PoolPrivate,MLS
30261,Active,Land,Gates Canyon Rd,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,vacaville,"[{'rating': ['7', '4', '6', '6', '10', '2'], '...",0,0,NaN,CA,NaN,380000,0,21904829
83522,New,Colonial,Cornejo Ricardo Descalzi,NaN,"{'atAGlanceFacts': [{'factValue': '1995', 'fac...",NaN,quito ecuador,"[{'rating': [], 'data': {'Distance': [], 'Grad...",NaN,0,NaN,NY,NaN,470000,0,3177007
231282,New construction,NaN,0 N Gopher Canyon Rd,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,bonsall,"[{'rating': ['7/10', '7/10', '3/10'], 'data': ...",NaN,--,449 acres,CA,NaN,60000000,0,oc19261036
235207,NaN,Townhouse,1744 N Dixie Hwy # 1744,3.0,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",NaN,fort lauderdale,"[{'rating': ['3/10', '5/10', '7/10'], 'data': ...","2,043",--,3,FL,NaN,425000,0,"1744, fort lauderdale, fl"
305572,for sale,lot/land,000 U.S. Hwy 359,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,laredo,"[{'rating': ['4/10'], 'data': {'Distance': ['7...","243,849 sqft",0,NaN,TX,NaN,1740000,0,20193508


In [132]:
# проверим наличие дублей при исключении из датасета того или иного поля
# columns = df.columns
# for column in columns:
#    print(column, ' - ',df.drop(columns=column, axis=1).duplicated().sum())